## Connecting to a DB

In [6]:
from mb.sql.conn import get_engine, get_session, get_base, get_metadata
from mb.sql.basic import read_sql,engine_execute,to_sql
import sqlalchemy as sa

In [7]:
engine = get_engine(name='postgresql' , db= 'postgres', user='postgres' , password= 'postgres', host= 'localhost', port= 5432, echo=False)
engine2 = get_engine(name='postgresql' , db= 'postgres_2', user='postgres' , password= 'postgres', host= 'localhost', port= 5432, echo=False)
session = get_session(engine)
base = get_base()
metadata= get_metadata(base,engine)

## Executing query functions in an engine

In [9]:
a = engine_execute(engine,'CREATE TABLE IF NOT EXISTS test3 (id serial PRIMARY KEY, num integer, data varchar);')

In [19]:
a1 = engine_execute(engine,'CREATE TABLE IF NOT EXISTS public.test1 (id serial PRIMARY KEY, num integer, data varchar);')

In [4]:
#b = engine_execute(conn,"INSERT INTO test3(id,num, data) VALUES (1,100, 'abc')")

In [20]:
list_tables(engine,schema='public')

,table_name
0,download_db
1,download_queue
2,merged_db
3,mix_video_db
4,test1
5,test3
6,transcript_db


In [21]:
c = engine_execute(engine,"SELECT * FROM test3;")
print(c.fetchall())

[]


In [22]:
a_sql = read_sql('SELECT * FROM test1;',engine)
a_sql

,id,num,data


In [23]:
a_sql2 = a_sql.copy()
to_sql(a_sql2,engine,'test4',schema='public',if_exists='replace',index=False)

In [24]:
q1 ="SELECT schema_name FROM information_schema.schemata WHERE schema_name NOT IN ('information_schema', 'mysql', 'performance_schema') ORDER BY schema_name;"
read_sql(q1,engine)

,schema_name
0,embeddingsDB
1,pg_catalog
2,pg_toast
3,public
4,ytshorts


## using extra functions

In [25]:
from mb.sql.utils import list_schemas,list_tables,rename_table,drop_table,drop_schema,create_schema,create_index,clone_db
from mb.utils.logging import logger

In [26]:
list_schemas(engine)

,schema_name
0,embeddingsDB
1,pg_catalog
2,pg_toast
3,public
4,ytshorts


In [27]:
list_tables(engine,schema='public')

,table_name
0,download_db
1,download_queue
2,merged_db
3,mix_video_db
4,test1
5,test3
6,test4
7,transcript_db


In [32]:
rename_table('test4_renamed','test4',engine)

In [33]:
list_tables(engine,schema='public')

,table_name
0,download_db
1,download_queue
2,merged_db
3,mix_video_db
4,test1
5,test3
6,test4_renamed
7,transcript_db


In [34]:
drop_table('test4_renamed',engine,logger=logger)

22:26:20 │ INFO     │ Table test4_renamed dropped.


In [35]:
list_tables(engine,schema='public')

,table_name
0,download_db
1,download_queue
2,merged_db
3,mix_video_db
4,test1
5,test3
6,transcript_db


In [36]:
create_schema('test_schema2',engine,logger=logger)

22:26:28 │ INFO     │ Schema test_schema2 created.


In [37]:
list_schemas(engine)

,schema_name
0,embeddingsDB
1,pg_catalog
2,pg_toast
3,public
4,test_schema2
5,ytshorts


In [38]:
drop_schema('test_schema2',engine,logger=logger)

22:26:32 │ INFO     │ Schema test_schema2 dropped.


In [39]:
list_schemas(engine)

,schema_name
0,embeddingsDB
1,pg_catalog
2,pg_toast
3,public
4,ytshorts


In [16]:
a = engine_execute(engine,'CREATE TABLE IF NOT EXISTS test_last_update (id serial PRIMARY KEY, num integer, data varchar,last_updated timestamp);')


In [17]:
a = '''INSERT INTO test_last_update (num, data, last_updated) VALUES
(1, 'foo', '2023-07-19 10:23:45'),
(2, 'bar', '2023-07-20 09:34:21'),
(3, 'baz', '2023-07-20 14:12:18'),
(4, 'qux', '2023-07-19 15:45:12'),
(5, 'quux', '2023-07-20 11:23:54'),
(6, 'corge', '2023-07-19 17:09:37'),
(7, 'grault', '2023-07-20 13:56:42'),
(8, 'garply', '2023-07-19 22:18:09'),
(9, 'waldo', '2023-07-20 08:47:33'),
(10, 'fred', '2023-07-19 16:37:28'),
(11, 'plugh', '2023-07-20 12:09:57'),
(12, 'xyzzy', '2023-07-19 19:22:13'),
(13, 'thud', '2023-07-20 10:01:04'),
(14, 'foo', '2023-07-19 14:55:39'),
(15, 'bar', '2023-07-20 15:27:16');'''

engine_execute(engine,a)


In [19]:
a = ''' CREATE TABLE IF NOT EXISTS test_last_update_2 (
  id serial PRIMARY KEY,
  num integer,
  data varchar,
  last_updated timestamp
);

INSERT INTO test_last_update_2 (num, data, last_updated) VALUES
  (1, 'foo', '2023-07-23 10:23:45'),
  (2, 'bar', '2023-07-24 09:34:21'),
  (3, 'baz', '2023-07-25 14:12:18'),
  (4, 'qux', '2023-07-26 15:45:12'),
  (5, 'quux', '2023-07-27 11:23:54'),
  (6, 'corge', '2023-07-28 17:09:37'),
  (7, 'grault', '2023-07-29 13:56:42'),
  (8, 'garply', '2023-07-30 22:18:09'),
  (9, 'waldo', '2023-08-01 08:47:33'),
  (10, 'fred', '2023-08-02 16:37:28'),
  (11, 'plugh', '2023-08-03 12:09:57'),
  (12, 'xyzzy', '2023-08-04 19:22:13'),
  (13, 'thud', '2023-08-05 10:01:04'),
  (14, 'foo', '2023-08-06 14:55:39'),
  (15, 'bar', '2023-08-07 15:27:16'); '''

engine_execute(engine,a)

In [ ]:
from mb.sql.update import get_last_updated_timestamp

In [31]:
import pandas as pd
def get_last_updated_timestamp(engine,table_name: str,schema : str ="public",updated_col: str = "updated") -> pd.Timestamp:
    """
    Gets the last updated timestamp from all records of an sqlite table containing the 'updated' field.

    Args:
        engine (sqlalchemy.engine.base.Engine): Engine object.
        table_name (str): tables containing the 'updated' field. 
        schema (str): Name of the schema. Default: None
        updated_col (str): name of the 'updated' field. Default: 'updated'
    Returns:
        ts (pandas.Timestamp): last updated timestamp
    """

    query_str = f"SELECT MAX({updated_col}) AS last_updated FROM {table_name};"
    df = read_sql(query_str,engine)

    ts = pd.Timestamp(df["last_updated"][0])
    if pd.isnull(ts):
        ts = pd.Timestamp("2014-01-01")
    return ts

In [36]:
k = get_last_updated_timestamp(engine,table_name='test_last_update',schema='public',updated_col='last_updated')